In [1]:
import numpy as np
import pandas as pd


import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image

In [2]:
# Load data set
movies=pd.read_csv('DataMovie/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users=pd.read_csv('DataMovie/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings=pd.read_csv('DataMovie/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# prepare training and test set

trainingset=pd.read_csv('DataMovie/ml-100k/u1.base', delimiter='\t', header=None)
testset=pd.read_csv('DataMovie/ml-100k/u1.test', delimiter='\t', header=None)

In [5]:
# convert to array as it is quicker 

trainingset=np.array(trainingset, dtype='int')
testset=np.array(testset, dtype='int')

In [6]:
# get total no. of movies and users in order to then make a matrix of the data


nb_users= int(max(max(trainingset[:,0]), max(testset[:,0])))
nb_movies=int(max(max(trainingset[:,1]), max(testset[:,1])))
print(nb_users, nb_movies)

943 1682


In [7]:
# make matrix of users in lines and movies in columns

def convert(data):
    new_data=[] #initialise list
    for id_users in range(1, nb_users+1):
        id_movies=data[:,1][data[:,0]==id_users]
        id_ratings=data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
trainingset=convert(trainingset)
testset=convert(testset)

In [9]:
# convert data into torch sensors

training_set = torch.FloatTensor(trainingset)
test_set=torch.FloatTensor(testset)

In [74]:
training_set


    1     1     1  ...     -1    -1    -1
    1    -1    -1  ...     -1    -1    -1
   -1    -1    -1  ...     -1    -1    -1
       ...          ⋱          ...       
    1    -1    -1  ...     -1    -1    -1
   -1    -1    -1  ...     -1    -1    -1
   -1     1    -1  ...     -1    -1    -1
[torch.FloatTensor of size 943x1682]

In [11]:
# convert ratings (1-5) into binary ratings 1 (liked) and 0 (not liked)

training_set[training_set==0] = -1 # not rated
training_set[training_set==1] = 0
training_set[training_set==2] = 0
training_set[training_set>=3] = 1

test_set[test_set==0] = -1 # not rated
test_set[test_set==1] = 0
test_set[test_set==2] = 0
test_set[test_set>=3] = 1


In [12]:
class RBM(nn.Module):
    def __init__(self,
                 n_vis=1682,
                 n_hin=600,
                 k=5):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hin,n_vis)*1e-2)
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hin))
        self.k = k
    
    def sample_from_p(self,p):
        return F.relu(torch.sign(p - Variable(torch.rand(p.size()))))
    
    def v_to_h(self,v):
        p_h = F.sigmoid(F.linear(v,self.W,self.h_bias))
        sample_h = self.sample_from_p(p_h)
        return p_h,sample_h
    
    def h_to_v(self,h):
        p_v = F.sigmoid(F.linear(h,self.W.t(),self.v_bias))
        sample_v = self.sample_from_p(p_v)
        return p_v,sample_v
    
   # def forward(self,v):
    #    pre_h1,h1 = self.v_to_h(v)
        
     #   h_ = h1
      #  for _ in range(self.k):
       #     pre_v_,v_ = self.h_to_v(h_)
        #    pre_h_,h_ = self.v_to_h(v_)
            
       # return v,v_, h_
    
    def free_energy(self,v):
        vbias_term = v.mv(self.v_bias)
        wx_b = F.linear(v,self.W,self.h_bias)
        hidden_term = wx_b.exp().add(1).log().sum(1)
        return (-hidden_term - vbias_term).mean()
        

In [22]:
rbm = RBM(k=1)
train_op = optim.SGD(rbm.parameters(),0.1)


batch_size_=32

for epoch in range(8):
    loss_ = []
    reconstruction_error=0
    s=0
    for n in range(0, len(training_set)- batch_size_, batch_size_):
        vk=training_set[n:n+batch_size_]
        vk=Variable(vk)
        
        v0=training_set[n:n+batch_size_]
        v0=Variable(v0)
              
        ph0,_=rbm.v_to_h(v0)
        for k in range(1):
            _,hk =rbm.v_to_h(vk)
            _,vk=rbm.h_to_v(hk)
            
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.v_to_h(vk)
        
        loss = rbm.free_energy(v0) - rbm.free_energy(vk)
        loss_.append(loss.data[0])
        train_op.zero_grad()
        
        loss.backward()
        train_op.step()
    
        reconstruction_error+=torch.mean(torch.abs(v0-vk))
        s+=1
    print ('Reconstructions error:  ' + str(reconstruction_error.data.numpy()[0]/s)+ 
          ' - free energy loss: ' + str(np.mean(loss_)))  

Reconstructions error:  0.0161874972541 - free energy loss: 0.748246817753
Reconstructions error:  0.0141719887997 - free energy loss: 1.70921167834
Reconstructions error:  0.0137004646762 - free energy loss: 1.2363628519
Reconstructions error:  0.0133602742491 - free energy loss: 1.10104685816
Reconstructions error:  0.012947690898 - free energy loss: 0.936798621868
Reconstructions error:  0.0128003399948 - free energy loss: 0.897213113719
Reconstructions error:  0.0125741876405 - free energy loss: 0.911114922885
Reconstructions error:  0.0124832134822 - free energy loss: 0.956266732051


In [117]:
# now predict on new observations 

test_loss=0                                       # need to measure error. loss function could use RMSE (done in autoencoders)
s=0 
vis1=[]
for id_user in range(0, nb_users): #batch learning
    v=test_set[id_user:id_user+ 1]  # training set inputs are used to activate neurons of our RBM
    vt=test_set[id_user:id_user + 1] #target
    
    v=Variable(v)
    vt=Variable(vt)
    if len(vt[vt>=0])>0:
        _,h =rbm.v_to_h(v)
        _,v=rbm.h_to_v(h)
        
        test_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))   #update train loss
        s+=1
    
    v=v.data.numpy()
    vis1.append(v)
print ('test loss: ' + str(test_loss/s))


test loss: Variable containing:
 0.3045
[torch.FloatTensor of size 1]



In [122]:
#sum(test_set.numpy()[4]==-1)
vis1=np.array(vis1)

In [123]:
#vis1=v.data.numpy()

In [189]:
vis0=test_set.numpy()



In [197]:


np.where(vis0[4]==0)

(array([ 68,  79, 109, 166, 224, 230, 233, 240, 242, 258, 363, 368, 369,
        375, 376, 387, 388, 392, 393, 396, 399, 401, 409, 410, 420, 423,
        438, 440, 443, 452, 453, 456]),)

In [200]:
vis1[4][0][240]

1.0

In [202]:
np.where(vis1[4][0]==0)

(array([   6,   12,   16,   17,   19,   20,   27,   31,   32,   33,   34,
          35,   38,   40,   42,   57,   67,   71,   74,   79,   80,   83,
          93,  102,  103,  104,  109,  114,  116,  118,  119,  127,  129,
         130,  138,  141,  145,  146,  148,  149,  152,  156,  157,  162,
         169,  174,  175,  176,  179,  185,  204,  216,  220,  222,  223,
         228,  230,  234,  239,  241,  243,  246,  248,  253,  255,  258,
         259,  262,  266,  277,  282,  285,  290,  293,  294,  295,  297,
         303,  304,  307,  308,  311,  313,  319,  321,  322,  323,  324,
         326,  330,  331,  334,  336,  337,  344,  346,  351,  354,  361,
         362,  363,  364,  367,  368,  372,  373,  374,  375,  382,  383,
         389,  390,  396,  397,  398,  399,  406,  408,  409,  411,  413,
         414,  416,  418,  419,  422,  423,  436,  437,  438,  439,  441,
         445,  446,  448,  449,  450,  451,  452,  453,  455,  456,  462,
         464,  467,  468,  490,  492, 